# DiCE Results on Credit Card Default

In [2]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from recourse_methods import dice_method
from models import regression
from core import recourse_iterator
from data.datasets.credit_card_default import credit_card_default_loader
from data.adapters import continuous_adapter
from models import model_utils
from core import utils
from visualize.two_d_plots import Display2DPaths

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the data and model

In [3]:
data_loader = credit_card_default_loader.CreditCardDefaultLoader()
dataset = data_loader.load_data()
preprocessor = continuous_adapter.StandardizingAdapter(perturb_ratio=0.1).fit(dataset)

model = regression.simple_model(dataset, preprocessor, 0)

X = preprocessor.transform(dataset).drop('Y', axis=1)
print(model.predict_proba(X))
print(model.predict(X))

[[0.55119158 0.44880842]
 [0.20273353 0.79726647]
 [0.14971968 0.85028032]
 ...
 [0.90796965 0.09203035]
 [0.16483424 0.83516576]
 [0.18284051 0.81715949]]
[-1  1  1 ... -1  1  1]


/home/jakeval/umass/research/.env/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/jakeval/umass/research/.env/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


## Generate some paths

Because DiCE generates counterfactual examples, the first direction it gives typically terminates in a valid CFE.

In [4]:
k_directions = 3
check_certainty = recourse_iterator.wrap_model(model, positive_index=1)
certainty_cutoff = 0.75

dice = dice_method.DiCE(
    k_directions,
    preprocessor,
    dataset,
    list(preprocessor.continuous_features),
    model,
    model_backend='sklearn',
    label_column='Y',
    dice_cfe_kwargs={'stopping_threshold': certainty_cutoff})

In [7]:
def run_trial(dice, preprocessor, certainty_cutoff, check_certainty):
    poi = utils.random_poi(dataset, column='Y', label = 1)
    iterator = recourse_iterator.RecourseIterator(
        dice,
        preprocessor,
        certainty_cutoff=certainty_cutoff,
        check_certainty=check_certainty)
    paths = iterator.iterate_k_recourse_paths(poi, 20)
    return paths

In [8]:
trials = 10

trial_paths = []

for i in range(trials):
    paths = run_trial(dice, preprocessor, certainty_cutoff, check_certainty)
    trial_paths.append(paths)

100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.10it/s]


In [9]:
def calculate_path_length(path):
    path = preprocessor.transform(path).to_numpy()
    length = 0
    for i in range(1, path.shape[0]):
        length += ((path[i] - path[i-1])**2).sum()**0.5
    return length

In [10]:
total_paths = 0
total_succeeded = 0
total_path_length = 0


for paths in trial_paths:
    for path in paths:
        total_paths += 1
        if check_certainty(path.iloc[-1]) > certainty_cutoff:
            total_succeeded += 1
        total_path_length += calculate_path_length(path)
        

print(total_succeeded / total_paths)
print(total_path_length / total_paths)

0.9666666666666667
53.17366265329683
